In [1]:
# Import libraries
from datetime import datetime
from datetime import timedelta

from pytz import timezone

from collections import defaultdict
from collections import Counter

from pprint import pprint

import pendulum
import pandas as pd

In [2]:
# Read data from file into list of list
df = pd.read_csv('data/cta_daily_summary_totals.csv', parse_dates=['service_date'])
print(df.head())

  service_date day_type     bus  rail_boardings  total_rides
0   2001-01-01        U  297192          126455       423647
1   2001-01-02        W  780827          501952      1282779
2   2001-01-03        W  824923          536432      1361355
3   2001-01-04        W  870021          550011      1420032
4   2001-01-05        W  890426          557917      1448343


In [3]:
dates_list = df.resample('50d', on='service_date', label='right', offset='-1d').sum().index.strftime('%m/%d/%Y').to_list()
df['service date'] = df.service_date.dt.strftime('%m/%d/%Y')
daily_summaries = list(df[['service date', 'day_type', 'bus', 'rail_boardings', 'total_rides']].to_records(index=False))

date_ranges_list = df.resample('30d', on='service_date', label='right', offset='-1d').sum().index.strftime('%m/%d/%Y').to_list()
date_ranges_str = list(zip(date_ranges_list[0::2], date_ranges_list[1::2]))
date_ranges_list = [datetime.strptime(d,'%m/%d/%Y') for d in date_ranges_list]
date_ranges = list(zip(date_ranges_list[0::2], date_ranges_list[1::2]))

In [4]:
df_rail = df.sort_values(by='service_date').groupby(['service_date', 'service date']).rail_boardings.sum().reset_index()
NY_ridership = df_rail[['service date', 'rail_boardings']].to_records(index = False)

#df_rail['service_date'].apply(lambda x: datetime(x.year, x.month, x.day))
#NY_ridership = df_rail.to_records(index=False)
# Get the year of a np.datetime
#print(NY_ridership[0][0])
#print(NY_ridership[0][0].astype('datetime64[Y]').astype(int) + 1970)

In [5]:
df = pd.read_csv('data/crime_sampler.csv', parse_dates=['Date'])
print(df.head())

                 Date                 Block Primary Type          Description  \
0 2016-05-23 17:35:00   024XX W DIVISION ST      ASSAULT               SIMPLE   
1 2016-03-26 20:20:00     019XX W HOWARD ST     BURGLARY       FORCIBLE ENTRY   
2 2016-04-25 15:05:00       001XX W 79TH ST        THEFT         RETAIL THEFT   
3 2016-04-26 17:30:00      010XX N PINE AVE      BATTERY               SIMPLE   
4 2016-06-19 01:15:00  027XX W AUGUSTA BLVD      BATTERY  AGGRAVATED: HANDGUN   

  Location Description  Arrest  Domestic  District  
0               STREET   False      True        14  
1   SMALL RETAIL STORE   False     False        24  
2     DEPARTMENT STORE    True     False         6  
3             SIDEWALK   False     False        15  
4             SIDEWALK   False     False        12  


In [6]:
parking_violations_dates = df[df['Location Description'].str.contains('PARKING', na=False, regex=False
                                                                     )].Date.dt.strftime('%m/%d/%Y').to_list()

In [7]:
df['day'] = df.Date.dt.strftime('%Y-%m-%d')
df['time'] = df.Date.dt.strftime('%H:%M:%S')
parking_violations = list(df[df['Location Description'].str.contains('PARKING', na=False,
                                                                     regex=False)][['day','time']].to_records(index = False))

# 04 Handling Dates and Times

Handling times can seem daunting at time, but here, you'll dig in and learn how to create datetime objects, print them, look to the past and to the future. Additionally, you'll learn about some third party modules that can make all of this easier. You'll continue to use the Chicago Transit dataset to answer questions about transit times.

## 04.01 There and Back Again a DateTime Journey

See the video.

In [8]:
# From string to datetime
parking_violations_date = '06/11/2016'

# Parsing strings into datetimes
date_dt = datetime.strptime(parking_violations_date, '%m/%d/%Y')
print(type(date_dt), date_dt)

# From Datetime to String
print(date_dt.strftime('%m/%d/%Y'))

# Printing a datetime as an ISO standard string
print(date_dt.isoformat())

<class 'datetime.datetime'> 2016-06-11 00:00:00
06/11/2016
2016-06-11T00:00:00


## 04.02 Strings to DateTimes

Time to begin your DateTime journey! You'll start by using the .strptime() method from the datetime object as shown in the video, passing it both the string and the format. A full list of the format string components is available in the Python documentation (https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior).

You'll be using the datetime column from the Chicago Transit Authority data, which is available as __dates_list__. Feel free to explore it in the IPython Shell: You'll see that it has the format of Month, Day, Year.

**Instructions**

1. Import the datetime object from datetime.
2. Iterate over the dates_list, using date_str as your iterator variable.
3. Convert each date_str into a datetime object called date_dt using the datetime.strptime() function, with '%m/%d/%Y' as your format.
4. Print each date_dt.

**Results:**<br>
<font color=darkgreen>Great work! You now know how to convert Strings to DateTimes. In the next exercise, you'll learn how to do the opposite!</font>

In [9]:
# Iterate over the dates_list 
for date_str in dates_list[:10]:
    # Convert each date to a datetime object: date_dt
    date_dt = datetime.strptime(date_str, '%m/%d/%Y')
    
    # Print each date_dt
    print(date_dt)

2001-02-19 00:00:00
2001-04-10 00:00:00
2001-05-30 00:00:00
2001-07-19 00:00:00
2001-09-07 00:00:00
2001-10-27 00:00:00
2001-12-16 00:00:00
2002-02-04 00:00:00
2002-03-26 00:00:00
2002-05-15 00:00:00


In [10]:
datetimes_list = []

# Iterate over the dates_list 
for date_str in dates_list:
    # Convert each date to a datetime object: date_dt
    date_dt = datetime.strptime(date_str, '%m/%d/%Y')
    
    datetimes_list.append(date_dt)

# Print first 10 items of datetimes_list
pprint(datetimes_list[:10])

[datetime.datetime(2001, 2, 19, 0, 0),
 datetime.datetime(2001, 4, 10, 0, 0),
 datetime.datetime(2001, 5, 30, 0, 0),
 datetime.datetime(2001, 7, 19, 0, 0),
 datetime.datetime(2001, 9, 7, 0, 0),
 datetime.datetime(2001, 10, 27, 0, 0),
 datetime.datetime(2001, 12, 16, 0, 0),
 datetime.datetime(2002, 2, 4, 0, 0),
 datetime.datetime(2002, 3, 26, 0, 0),
 datetime.datetime(2002, 5, 15, 0, 0)]


## 04.03 Converting to a String

Converting from a datetime object to a string is done with the __.strftime()__ method on a instance of the datetime object. You pass a format string just like the ones used in the prior exercise.

There is also a widely used string output standard called ISO-8601. It has a shortcut method named __.isoformat()__. I encourage you to use it anytime you write out to a file.

All the datetimes you created for the transit data in the prior exercise are saved in the __datetimes_list__.

**Instructions**

1. Loop over the first 10 items of the datetimes_list, using item as your iterator variable.
2. Print out the item as a string in the format of 'MM/DD/YYYY'. For this, the format string is '%m/%d/%Y'.
3. Print out the item as an ISO standard string.

**Results:**<br>
<font color=darkgreen>Nicely done!</font>

In [11]:
# Loop over the first 10 items of the datetimes_list
for item in datetimes_list[:10]:
    # Print out the record as a string in the format of 'MM/DD/YYYY'
    print(item.strftime('%m/%d/%Y'))
    
    # Print out the record as an ISO standard string
    print(item.isoformat())

02/19/2001
2001-02-19T00:00:00
04/10/2001
2001-04-10T00:00:00
05/30/2001
2001-05-30T00:00:00
07/19/2001
2001-07-19T00:00:00
09/07/2001
2001-09-07T00:00:00
10/27/2001
2001-10-27T00:00:00
12/16/2001
2001-12-16T00:00:00
02/04/2002
2002-02-04T00:00:00
03/26/2002
2002-03-26T00:00:00
05/15/2002
2002-05-15T00:00:00


## 04.04 Working with Datetime Components and current time

See the video.

In [12]:
print(parking_violations_dates[:4])
daily_violations = defaultdict(int)

for violation in parking_violations_dates:
    violation_date = datetime.strptime(violation, '%m/%d/%Y')
    daily_violations[violation_date.day] += 1

print(daily_violations)

['07/30/2016', '05/24/2016', '08/17/2016', '02/25/2016']
defaultdict(<class 'int'>, {30: 16, 24: 27, 17: 17, 25: 16, 5: 18, 29: 16, 3: 16, 8: 19, 21: 17, 4: 20, 12: 21, 14: 20, 26: 20, 13: 22, 2: 21, 31: 12, 6: 16, 11: 22, 27: 20, 9: 16, 28: 17, 18: 22, 16: 24, 7: 22, 10: 15, 20: 17, 1: 21, 23: 19, 19: 20, 15: 22, 22: 19})


## 04.05 Pieces of Time

When working with datetime objects, you'll often want to group them by some component of the datetime such as the month, year, day, etc. Each of these are available as attributes on an instance of a __datetime__ object.

You're going to work with the summary of the CTA's daily ridership. It contains the following columns, in order: __service_date__, __day_type__, __bus__, __rail_boardings__, and __total_rides__. The modules __defaultdict__ and __datetime__ have already been imported for you.

**Instructions**

1. Create a defaultdict of an integer called monthly_total_rides.
2. Loop over the list daily_summaries, which contains the columns mentioned above in the assignment text.
3. Convert the service_date (1st element of daily_summary) to a datetime object called service_datetime. Use '%m/%d/%Y' as your format string.
4. Use the month of the service_datetime as the dict key and add the total_rides (5th element of daily_summary) to the current amount for the month. Be sure to convert this into an integer.
5. Print monthly_total_rides.

**Results:**<br>
<font color=darkgreen>Excellent work! Which month had the highest ridership?</font>

In [13]:
# Create a defaultdict of an integer: monthly_total_rides
monthly_total_rides = defaultdict(int)

# Loop over the list daily_summaries
for daily_summary in daily_summaries:
    # Convert the service_date to a datetime object
    service_datetime = datetime.strptime(daily_summary[0], '%m/%d/%Y')

    # Add the total rides to the current amount for the month
    monthly_total_rides[service_datetime.month] += int(daily_summary[4])
    
# Print monthly_total_rides
pprint(monthly_total_rides)

defaultdict(<class 'int'>,
            {1: 515062454,
             2: 500276873,
             3: 557894281,
             4: 544878980,
             5: 564403630,
             6: 553707053,
             7: 552970459,
             8: 558434623,
             9: 574770898,
             10: 652199892,
             11: 538491629,
             12: 500560093})


In [14]:
# Create a Counter: monthly_total_rides
monthly_total_rides = Counter([datetime.strptime(daily_summary[0], '%m/%d/%Y').month for daily_summary in daily_summaries])
print(monthly_total_rides.most_common())

[(10, 434), (1, 403), (3, 403), (5, 403), (7, 403), (8, 403), (12, 403), (4, 390), (6, 390), (9, 390), (11, 390), (2, 367)]


## 04.06 Creating DateTime Objects... Now

Often when working with __datetime__ objects, you'll want to work on windows or ranges that start from the current date and time. You can do this using the __datetime__ now functions. There is a __.now()__ method on the __datetime__ object in the __datetime__ module and a __.utcnow()__ method. The __.now()__ method returns the current local time on the machine on which it is run, and __.utcnow()__ does the same thing but returns the value in UTC time. You'll need to be very familiar with these methods.

No dataset is used in this exercise, but bear with us as you'll need to do this often to compare year/month-to-date etc.

**Instructions**

1. Import datetime from the datetime module.
2. Store the local datetime as local_dt and print it.
3. Store the UTC datetime as utc_dt and print it.

**Results:**<br>
<font color=darkgreen>Well done! These are very useful methods to know.</font>

In [15]:
# Compute the local datetime: local_dt
local_dt = datetime.now()

# Print the local datetime
print(local_dt)

# Compute the UTC datetime: utc_dt
utc_dt = datetime.utcnow()

# Print the UTC datetime
print(utc_dt)

2021-02-17 21:47:26.840596
2021-02-18 03:47:26.841595


## 04.07 Timezones

In order to work effectively with other timezones, you can use the __pytz__ library. To use timezones, you need to import the __timezone__ object from the __pytz__ module. Then you can use the __timezone__ constructor and pass it a name of a timezone, such as __CT = timezone('US/Central')__. You can get a full list of timezone names at Wikipedia (https://en.wikipedia.org/wiki/List_of_tz_database_time_zones). In Python 3, you can make a datetime object "aware" by passing a timezone as the __tzinfo__ keyword argument to the __.replace()__ method on a datetime instance.

An "aware" datetime object has an __.astimezone()__ method that accepts a timezone object and returns a new datetime object in the desired timezone. If the tzinfo is not set for the datetime object it assumes the timezone of the computer you are working on.

A list, __daily_summaries__, has been supplied for you it contains the __datetime__ and rail ridership for trains going to New York. You need to determine the time in New York so you can align it with the New York Transit Authority data.

**Instructions**

1. Create a Timezone object for Chicago ('US/Central') called chicago_usa_tz.
2. Create a Timezone object for New York ('US/Eastern') called ny_usa_tz.
3. Iterate over the daily_summaries, unpacking it into the variables orig_dt and ridership.
4. Make the orig_dt timezone "aware" for Chicago, using chicago_usa_tz. Store the result in chicago_dt.
5. Convert chicago_dt to the New York timezone, ny_dt.
5. Print the chicago_dt, ny_dt, and ridership.

**Results:**<br>
<font color=darkgreen>Super!</font>

In [16]:
# Print total_rides
pprint(NY_ridership[:2])

# Create a defaultdict of an integer: total_rides
total_rides = []

# Loop over the list daily_summaries
for daily_summary in NY_ridership:
    # Add the total rides to the current amount for date
    total_rides.append((datetime.strptime(daily_summary[0], '%m/%d/%Y'), daily_summary[1]))
    
# Print total_rides
pprint(total_rides[:2])

rec.array([('01/01/2001', 126455), ('01/02/2001', 501952)],
          dtype=[('service date', 'O'), ('rail_boardings', '<i8')])
[(datetime.datetime(2001, 1, 1, 0, 0), 126455),
 (datetime.datetime(2001, 1, 2, 0, 0), 501952)]


In [17]:
# Create a Timezone object for Chicago
chicago_usa_tz = timezone('US/Central')

# Create a Timezone object for New York
ny_usa_tz = timezone('US/Eastern')

for orig_dt, ridership in total_rides[:10]:

    # Make the orig_dt timezone "aware" for Chicago
    chicago_dt = orig_dt.replace(tzinfo=chicago_usa_tz)
    
    # Convert chicago_dt to the New York Timezone
    ny_dt = chicago_dt.astimezone(ny_usa_tz)
    
    # Print the chicago_dt, ny_dt, and ridership
    print('Chicago: %s, NY: %s, Ridership: %s' % (chicago_dt, ny_dt, ridership))

Chicago: 2001-01-01 00:00:00-05:51, NY: 2001-01-01 00:51:00-05:00, Ridership: 126455
Chicago: 2001-01-02 00:00:00-05:51, NY: 2001-01-02 00:51:00-05:00, Ridership: 501952
Chicago: 2001-01-03 00:00:00-05:51, NY: 2001-01-03 00:51:00-05:00, Ridership: 536432
Chicago: 2001-01-04 00:00:00-05:51, NY: 2001-01-04 00:51:00-05:00, Ridership: 550011
Chicago: 2001-01-05 00:00:00-05:51, NY: 2001-01-05 00:51:00-05:00, Ridership: 557917
Chicago: 2001-01-06 00:00:00-05:51, NY: 2001-01-06 00:51:00-05:00, Ridership: 255356
Chicago: 2001-01-07 00:00:00-05:51, NY: 2001-01-07 00:51:00-05:00, Ridership: 169825
Chicago: 2001-01-08 00:00:00-05:51, NY: 2001-01-08 00:51:00-05:00, Ridership: 590706
Chicago: 2001-01-09 00:00:00-05:51, NY: 2001-01-09 00:51:00-05:00, Ridership: 599905
Chicago: 2001-01-10 00:00:00-05:51, NY: 2001-01-10 00:51:00-05:00, Ridership: 602052


## 04.08 Time Travel (Adding and Subtracting Time)

See the video.

In [18]:
flashback = timedelta(days=90)
print('Date 1:', orig_dt)
print('Date 2:', local_dt)

# Subtracting timedeltas
print('Data 1 - 90 días:', orig_dt - flashback)

# Adding timedeltas
print('Data 1 + 90 días', orig_dt + flashback)

# Datetime differences
time_diff = local_dt - orig_dt
print('Difference between dates:', type(time_diff))
print(time_diff)

Date 1: 2001-01-10 00:00:00
Date 2: 2021-02-17 21:47:26.840596
Data 1 - 90 días: 2000-10-12 00:00:00
Data 1 + 90 días 2001-04-10 00:00:00
Difference between dates: <class 'datetime.timedelta'>
7343 days, 21:47:26.840596


## 02.09 Finding a time in the future and from the past

Another common case when working with times is to get a date 30, 60, or 90 days in the past from some date. In Python, the __timedelta__ object from the __datetime__ module is used to represent differences in __datetime__ objects. You can create a __timedelta__ by passing any number of keyword arguments such as days, seconds, microseconds, milliseconds, minutes, hours, and weeks to __timedelta()__.

Once you have a __datetime__ object, you can add or subtract it from a __datetime__ object to get a __datetime__ object relative to the original datetime object.

A dictionary, __daily_summaries__, has been supplied for you. It contains the __datetime__ as the key with a dict as the value that has __'day_type'__ and __'total_ridership'__ keys. A list of datetimes to review called review_dates is also available.

**Instructions**

1. Import timedelta from the datetime module.
2. Build a timedelta of 30 days called glanceback using timedelta().
3. Iterate over the review_dates, using date as your iterator variable.
4. Calculate the date 30 days back by subtracting glanceback from date.
5. Print the date, along with 'day_type' and 'total_ridership' from daily_summaries for that date.
6. Print the prior_period_dt, along with 'day_type' and 'total_ridership' from daily_summaries for that date (prior_period_dt).

**Results:**<br>
<font color=darkgreen>Well done!</font>

In [19]:
# Create the variables
review_dates = [datetime(2013, 12, 22, 0, 0),
                datetime(2013, 12, 23, 0, 0),
                datetime(2013, 12, 24, 0, 0),
                datetime(2013, 12, 25, 0, 0),
                datetime(2013, 12, 26, 0, 0),
                datetime(2013, 12, 27, 0, 0),
                datetime(2013, 12, 28, 0, 0),
                datetime(2013, 12, 29, 0, 0),
                datetime(2013, 12, 30, 0, 0),
                datetime(2013, 12, 31, 0, 0)]

new_ds = {datetime.strptime(d, '%m/%d/%Y'): {'day_type': dt, 'total_ridership': tr} for d, dt, _, _, tr in daily_summaries}

In [20]:
# Build a timedelta of 30 days: glanceback
glanceback = timedelta(days=30)

# Iterate over the review_dates as date
for date in review_dates:
    # Calculate the date 30 days back: prior_period_dt
    prior_period_dt = date - glanceback
    
    # Print the review_date, day_type and total_ridership
    print('Date: %s, Type: %s, Total Ridership: %s' %
         (date, 
          new_ds[date]['day_type'], 
          new_ds[date]['total_ridership']))

    # Print the prior_period_dt, day_type and total_ridership
    print('Date: %s, Type: %s, Total Ridership: %s' %
         (prior_period_dt, 
          new_ds[prior_period_dt]['day_type'], 
          new_ds[prior_period_dt]['total_ridership']))

Date: 2013-12-22 00:00:00, Type: U, Total Ridership: 685457
Date: 2013-11-22 00:00:00, Type: W, Total Ridership: 1752614
Date: 2013-12-23 00:00:00, Type: W, Total Ridership: 1236510
Date: 2013-11-23 00:00:00, Type: A, Total Ridership: 1048943
Date: 2013-12-24 00:00:00, Type: W, Total Ridership: 815873
Date: 2013-11-24 00:00:00, Type: U, Total Ridership: 674817
Date: 2013-12-25 00:00:00, Type: U, Total Ridership: 363078
Date: 2013-11-25 00:00:00, Type: W, Total Ridership: 1641025
Date: 2013-12-26 00:00:00, Type: W, Total Ridership: 995622
Date: 2013-11-26 00:00:00, Type: W, Total Ridership: 1681213
Date: 2013-12-27 00:00:00, Type: W, Total Ridership: 1191650
Date: 2013-11-27 00:00:00, Type: W, Total Ridership: 1441786
Date: 2013-12-28 00:00:00, Type: A, Total Ridership: 911223
Date: 2013-11-28 00:00:00, Type: U, Total Ridership: 554312
Date: 2013-12-29 00:00:00, Type: U, Total Ridership: 627779
Date: 2013-11-29 00:00:00, Type: W, Total Ridership: 1074544
Date: 2013-12-30 00:00:00, Type:

## 02.10 Finding differences in DateTimes

Just like you were able to subtract a __timedelta__ from a __datetime__ to find a date in the past, you can also calculate the difference between two dates to get the __timedelta__ between in return. Here, you'll find out how much time has elapsed between two transit dates.

A list of tuples called __timedelta__ is provided for you. We took the dates from our dataset at every 30th record, and we paired up the records into tuples in a stepwise fashion.

**Instructions**

1. Iterate over date_ranges, unpacking it into start_date and end_date.
2. Print the end_date and start_date using the same print() function.
3. Print the difference between each end_date and start_date.

**Results:**<br>
<font color=darkgreen>Excellent! As expected, the difference between each end_date and start_date is 30 days.</font>

In [21]:
# Iterate over the date_ranges
for start_date, end_date in date_ranges[:5]:
    # Print the End and Start Date
    print(end_date, start_date)
    # Print the difference between each end and start date
    print(end_date - start_date)

2001-03-01 00:00:00 2001-01-30 00:00:00
30 days, 0:00:00
2001-04-30 00:00:00 2001-03-31 00:00:00
30 days, 0:00:00
2001-06-29 00:00:00 2001-05-30 00:00:00
30 days, 0:00:00
2001-08-28 00:00:00 2001-07-29 00:00:00
30 days, 0:00:00
2001-10-27 00:00:00 2001-09-27 00:00:00
30 days, 0:00:00


## 04.11 HELP! Libraries to make it easier

See the video.

In [22]:
# Parsing time with pendulum
occurred = parking_violations[0][0] + ' ' + parking_violations[0][1]
print('In:', occurred)

occurred_dt = pendulum.parse(occurred, tz='US/Eastern')
print('Pendulum type (US/Eastern):', occurred_dt)

# Timezone hopping with pendulum
print('Pendulum type (US/Central):', occurred_dt.in_timezone('US/Central'))
print('Pendulum type (Asia/Tokyo):', occurred_dt.in_timezone('Asia/Tokyo'))
print('Now             :', pendulum.now()) #always be UTC
print('Now (US/Eastern):', pendulum.now('US/Eastern'))
print('Now (Asia/Tokyo):', pendulum.now('Asia/Tokyo')) 

# Humanizing differences
now_day = pendulum.now()
diff = now_day - occurred_dt
print(diff)
print(diff.in_years(), 'years')
print(diff.in_months(), 'months')
print(diff.in_days(), 'days')
print(diff.in_hours(), 'hours')
print(diff.in_words())

# Set spanish
pendulum.set_locale('es')
print(diff.in_words())

In: 2016-07-30 20:34:00
Pendulum type (US/Eastern): 2016-07-30T20:34:00-04:00
Pendulum type (US/Central): 2016-07-30T19:34:00-05:00
Pendulum type (Asia/Tokyo): 2016-07-31T09:34:00+09:00
Now             : 2021-02-17T21:47:27.999245-06:00
Now (US/Eastern): 2021-02-17T22:47:27.999245-05:00
Now (Asia/Tokyo): 2021-02-18T12:47:27.999245+09:00
<Period [2016-07-30T20:34:00-04:00 -> 2021-02-17T21:47:27.999245-06:00]>
4 years
54 months
1663 days
39915 hours
4 years 6 months 2 weeks 4 days 3 hours 13 minutes 27 seconds
4 años 6 meses 2 semanas 4 días 3 horas 13 minutos 27 segundos


## 04.12 Localizing time with pendulum

Here, you're going to use __pendulum__ to practice doing some common datetime operations!

**Instructions**

1. Import the pendulum module.
2. Create a now datetime for Tokyo ('Asia/Tokyo') called tokyo_dt.
3. Convert tokyo_dt to Los Angeles time ('America/Los_Angeles'). Store the result as la_dt.
4. Print the ISO 8601 string of la_dt, using the .to_iso8601_string() method.

**Results:**<br>
<font color=darkgreen>Great work!</font>

In [23]:
# Create a now datetime for Tokyo: tokyo_dt
tokyo_dt = pendulum.now('Asia/Tokyo')
print(tokyo_dt)
print(tokyo_dt.to_iso8601_string())

# Covert the tokyo_dt to Los Angeles: la_dt
la_dt = tokyo_dt.in_timezone('America/Los_Angeles')

# Print the ISO 8601 string of la_dt
print(la_dt)
print(la_dt.to_iso8601_string())

2021-02-18T12:47:28.114503+09:00
2021-02-18T12:47:28.114503+09:00
2021-02-17T19:47:28.114503-08:00
2021-02-17T19:47:28.114503-08:00


## 04.13 Humanizing Differences with Pendulum

Pendulum provides a powerful way to convert strings to __pendulum datetime__ objects via the __.parse()__ method. Just pass it a date string and it will attempt to convert into a valid __pendulum datetime__. By default, __.parse()__ can process dates in ISO 8601 format. To allow it to parse other date formats, pass __strict = False__.

It also has a wonderful alternative to __timedelta__. When calculating the difference between two dates by subtraction, __pendulum__ provides methods such as __.in_days()__ to output the difference in a chosen metric. These are just the beginning of what pendulum can do for you.

A list of tuples called __date_ranges__ is provided for you. This is the same list of tuples that contain two dates that was used a few exercises prior. You'll be focusing on comparing ranges of records.

You can learn more in the pendulum documentation (https://pendulum.eustace.io/docs/). Here, it has been imported for you.

**Instructions**

1. Iterate over the date_ranges list, unpacking it into start_date and end_date. These dates are not in ISO 8601 format.
2. Use pendulum to convert the start_date string to a pendulum date called start_dt.
3. Use pendulum to convert the end_date string to pendulum date called end_dt.
4. Calculate the difference between end_dt and start_dt. Store the result as diff_period.
5. Print the difference in days, using the .in_days() method.

**Results:**<br>
<font color=darkgreen>Wonderful work! Time to cement your newly acquired skills in a case study. See you in Chapter 5!</font>

In [24]:
# Iterate over date_ranges
for start_date, end_date in date_ranges_str[:5]:

    # Convert the start_date string to a pendulum date: start_dt 
    start_dt = pendulum.parse(start_date, strict = False)
    
    # Convert the end_date string to a pendulum date: end_dt 
    end_dt = pendulum.parse(end_date, strict = False)
    
    # Print the End and Start Date
    print(end_dt, start_dt)
    
    # Calculate the difference between end_dt and start_dt: diff_period
    diff_period = end_dt - start_dt
    
    # Print the difference in days
    print(diff_period.in_days())

2001-03-01T00:00:00+00:00 2001-01-30T00:00:00+00:00
30
2001-04-30T00:00:00+00:00 2001-03-31T00:00:00+00:00
30
2001-06-29T00:00:00+00:00 2001-05-30T00:00:00+00:00
30
2001-08-28T00:00:00+00:00 2001-07-29T00:00:00+00:00
30
2001-10-27T00:00:00+00:00 2001-09-27T00:00:00+00:00
30


# Aditional material

- **Datacamp course**: https://learn.datacamp.com/courses/data-types-for-data-science-in-python
- **Python's strftime directives**: 
> https://strftime.org/ <br>
> https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior
- **Time Zone Database (Olsen Database)**: http://www.iana.org/time-zones
- **Pendulum documentation**: https://pendulum.eustace.io/docs/